## Conexão Banco de dados Postgres


In [12]:
import os
import pandas as pd
import urllib.parse
from dotenv import load_dotenv
from sqlalchemy import create_engine, text


engine = create_engine(conn_url)

try:
    with engine.connect() as conn:
        df_churn_anual = pd.read_sql(text(sql_query), conn)

    print("✅ Sucesso! Os dados foram carregados em 'df_churn_anual'.")
    print(df_churn_anual.head())

except Exception as e:
    print(f" Erro ao executar consulta: {e}")

# 1. Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# 2. Configuração das credenciais com tratamento para caracteres especiais (como o '@')
# Mude apenas esta parte do código anterior:
user     = os.getenv("PROD_POSTGRES_USERNAME")
password = urllib.parse.quote_plus(os.getenv("PROD_POSTGRES_PASSWORD"))
host     = os.getenv("PROD_POSTGRES_HOST")
database = os.getenv("PROD_POSTGRES_DATABASE")
port     = os.getenv("POSTGRES_PORT", "5432") # A porta costuma ser a mesma
# 3. Construção da URL de conexão e criação do Engine
conn_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(conn_url)

# 4. Definição da sua Query SQL de Churn Anual
sql_query = """
with base_contratos as (
    select
        distinct
        c.nr_central_sigma,
        ccc.codigo_contrato,
        ccc.data_pedido as data_inicio,
        ccc.data_rescisao,
        (
            select mcc.situacao
            from movimento_contratos_cli mcc
            where mcc.codigo_contrato = ccc.codigo_contrato
              and mcc.situacao <> 7
            order by mcc.data_conclusao desc
            limit 1
        ) as situacao_movimento
    from clientes c
    inner join cab_contratos_cli ccc on ccc.codigo_cliente = c.codigo_cliente
),

clientes_com_contrato_ativo as (
    select distinct nr_central_sigma
    from base_contratos
    where data_rescisao is null
),

churn_por_cliente as (
    select
        bc.nr_central_sigma,
        MAX(bc.data_rescisao) as data_churn
    from base_contratos bc
    where bc.data_rescisao is not null
      and bc.nr_central_sigma not in (select nr_central_sigma from clientes_com_contrato_ativo)
    group by bc.nr_central_sigma
),

data_minima as (
    select date_trunc('year', min(data_inicio)) as inicio_historico 
    from base_contratos
),

churn_anual as (
    select
        DATE_TRUNC('year', ch.data_churn) as ano_churn,
        COUNT(distinct ch.nr_central_sigma) as clientes_churn
    from churn_por_cliente ch
    group by 1
),

clientes_ativos_inicio_ano as (
    select
        sa.ano,
        COUNT(distinct bc.nr_central_sigma) as total_clientes_inicio
    from (
        select generate_series(
            (select inicio_historico from data_minima),
            CURRENT_DATE,
            '1 year'
        )::date as ano
    ) sa
    cross join base_contratos bc
    where bc.data_inicio < sa.ano
      and (bc.data_rescisao is null or bc.data_rescisao >= sa.ano)
    group by sa.ano
)

select
    TO_CHAR(caa.ano, 'YYYY') as ano,
    caa.total_clientes_inicio as clientes_no_inicio_do_ano,
    coalesce(can.clientes_churn, 0) as total_churn_no_ano,
    case
        when caa.total_clientes_inicio = 0 then 0
        else ROUND((coalesce(can.clientes_churn, 0)::numeric / caa.total_clientes_inicio::numeric) * 100, 2)
    end as churn_rate_anual_percentual,
    case 
        when (coalesce(can.clientes_churn, 0)::numeric / nullif(caa.total_clientes_inicio,0)::numeric) * 100 > 15 then 'CRÍTICO (Acima de 15% ano)'
        when (coalesce(can.clientes_churn, 0)::numeric / nullif(caa.total_clientes_inicio,0)::numeric) * 100 > 10 then 'ALERTA'
        else 'SAUDÁVEL'
    end as status_saude
from clientes_ativos_inicio_ano caa
left join churn_anual can on caa.ano = can.ano_churn
where caa.total_clientes_inicio > 0 
order by caa.ano;
"""

# 5. Execução e salvamento no DataFrame df_churn_anual
try:
    df_churn_anual = pd.read_sql(sql_query, engine)
    print(" Sucesso! Os dados foram carregados em 'df_churn_anual'.")
    
    # Exibir as primeiras linhas para conferência
    print(df_churn_anual.head())

except Exception as e:
    print(f" Erro ao executar consulta: {e}")

✅ Sucesso! Os dados foram carregados em 'df_churn_anual'.
    ano  clientes_no_inicio_do_ano  total_churn_no_ano  \
0  2004                         80                   0   
1  2005                        371                  88   
2  2006                        782                 186   
3  2007                       1125                 235   
4  2008                       1470                 331   

   churn_rate_anual_percentual                status_saude  
0                         0.00                    SAUDÁVEL  
1                        23.72  CRÍTICO (Acima de 15% ano)  
2                        23.79  CRÍTICO (Acima de 15% ano)  
3                        20.89  CRÍTICO (Acima de 15% ano)  
4                        22.52  CRÍTICO (Acima de 15% ano)  
 Erro ao executar consulta: sqlalchemy.cyextension.immutabledict.immutabledict is not a sequence


In [13]:
df_churn_anual

,ano,clientes_no_inicio_do_ano,total_churn_no_ano,churn_rate_anual_percentual,status_saude
0,2004,80,0,0.00,SAUDÁVEL
1,2005,371,88,23.72,CRÍTICO (Acima de 15% ano)
2,2006,782,186,23.79,CRÍTICO (Acima de 15% ano)
3,2007,1125,235,20.89,CRÍTICO (Acima de 15% ano)
4,2008,1470,331,22.52,CRÍTICO (Acima de 15% ano)
5,2009,1793,346,19.30,CRÍTICO (Acima de 15% ano)
6,2010,2021,355,17.57,CRÍTICO (Acima de 15% ano)
7,2011,2307,485,21.02,CRÍTICO (Acima de 15% ano)
8,2012,2828,540,19.09,CRÍTICO (Acima de 15% ano)
9,2013,3458,702,20.30,CRÍTICO (Acima de 15% ano)
